In [1]:
import pandas as pd
import numpy as np
import os
from datetime import date
from ailab_tools.smn import encrypted_id_to_smn_uid
from ailab_tools.utils import GMail
from ailab_tools.utils import GSheet
from ailab_tools.utils import python_utils as pu

from ailab_tools.magics import slack_notify

Qlifeデータのsmn_uidへの変換<br>
出力結果はsegment_idごとのsmn_uidリストtsv<br>

ファイル名を入れてRun Allでおｋ<br>

<具体的な処理の流れ><br>
0.対象ファイル名をセット<br>
1.ファイル読み込み<br>
2.smn_uidへの変換<br>
3.ファイル出力<br>

In [2]:
# 0.対象ファイル名をセット
file_name = "20190731120317_user_segment.tsv"

In [3]:
# 1.ファイル読み込み
file_dir = f"/home/yusuke_akada/notebook/qlife/input/{file_name}"
df = pd.read_csv(file_dir, sep = "\t" )
df.head()

,logicad_id,segment_id,weight,expiration,action
0,ASu-kRHdaiC9ks8AAyiKcjQ5oM0n2A,allergy,0,2017-12-26T00:00+09:00,add
1,AYJzgzotO-9qks8AApYctf-Mic0n2A,allergy,0,2017-12-26T00:00+09:00,add
2,AXSWxicjpmdBks8ABbAZ30fNic0n2A,allergy,0,2017-12-26T00:00+09:00,add
3,AdIUZGB1H3rwks8ABXXkup4fu80n2A,allergy,0,2017-12-26T00:00+09:00,add
4,AYiM16OpZGbEks8ABKH7KNRVYc0n2A,allergy,0,2017-12-26T00:00+09:00,add


In [4]:
%%slack_notify @yusuke_akada
# 2.smn_uidへの変換


# 例外データ確認用
# encrypted_id_to_smn_uidで変換できない値が稀にあるのでそれを摘出する
except_idx = []
for idx, row in df.iterrows():
    try:
        encrypted_id_to_smn_uid([row.logicad_id])
    except:
        except_idx.append(idx)

# 例外データ
except_df = df[(df.index.isin(except_idx))]
except_id = list(except_df.logicad_id)


# 例外データを除いたDataFrameを用意
tmp_df = df[~df.logicad_id.isin(except_id)]

# smn_uidへの変換
dmpuser_list = list(tmp_df.logicad_id)
uid_list = encrypted_id_to_smn_uid(dmpuser_list)

# 変換したsmn_uidと元のデータを紐づけ
uid_df = pd.DataFrame({"smn_uid" :uid_list})
df_ = pd.concat([tmp_df, uid_df], axis=1 )
df_ = df_[(~df_.smn_uid.isnull())]
df_ = df_.astype({"smn_uid":int})

INFO:ailab_tools.utils.slack:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.utils.slack:Slack: Posted a message to Slack.


In [5]:
except_df

,logicad_id,segment_id,weight,expiration,action
778520,ghostery,obgyn,0,2017-12-26T00:00+09:00,add


In [6]:
# segment_idごとのuu数はこんな感じ
df_.groupby("segment_id").smn_uid.nunique()

segment_id
allergy                 3582
arthritis               1022
atopy                  15132
body_odor                470
cancer                  3155
cholesterol            56640
chronic_fatigue         1775
constipation          106543
diabetes               11216
diabetes_diet           2577
diet                    2342
ed                       310
eye_decay               8544
eye_disease            18461
eye_fatigue             7838
fatigue                   59
fatigue_liver            357
heart_disease          45477
hypertension           18306
joint_pain              1195
lifestyle_diseases     33941
menopause              17229
metabolic              71597
obgyn                 851326
scalp_care              1305
shoulder_back_pain      2171
skin_care             177460
skin_trouble           13101
thinning_hair           2719
Name: smn_uid, dtype: int64

In [7]:
%%slack_notify @yusuke_akada
# 3.ファイル出力


# ファイル出力用のディレクトリ作成
today_yymmdd = date.today().strftime("%y%m%d")
dir_name = f"/home/yusuke_akada/notebook/qlife/output/{today_yymmdd}" 
os.mkdir(dir_name)

contents = set(list(df_.segment_id))

# セグメントごとに処理
for x in contents:
    
    # ファイル出力
    output_file = f"{dir_name}/qlife_{x}.tsv" 
    segment_user = df_[(df_.segment_id == x)].smn_uid
    segment_user.to_csv(output_file, sep = "\t", index = False, header = False )

INFO:ailab_tools.utils.slack:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.utils.slack:Slack: Posted a message to Slack.
